Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
def calculate_logits(tf_train_dataset, weights0, weights1, biases0, biases1):
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights0) + biases0)
    output_layer = tf.matmul(hidden_layer, weights1) + biases1
    return output_layer
    
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights0 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases0 = tf.Variable(tf.zeros([1024]))
  weights1 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  beta = tf.constant(0.01)


  # Training computation.
  logits = calculate_logits(tf_train_dataset, weights0, weights1, biases0, biases1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
       + beta*tf.nn.l2_loss(weights0)+ beta*tf.nn.l2_loss(weights1)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(calculate_logits(tf_valid_dataset, weights0, weights1, biases0, biases1))
  test_prediction = tf.nn.softmax(calculate_logits(tf_test_dataset, weights0, weights1, biases0, biases1)) 
    

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3544.900879
Minibatch accuracy: 7.0%
Validation accuracy: 34.7%
Minibatch loss at step 500: 21.215445
Minibatch accuracy: 88.3%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 0.865051
Minibatch accuracy: 82.8%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.752743
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.735271
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.809304
Minibatch accuracy: 78.9%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.881230
Minibatch accuracy: 78.9%
Validation accuracy: 83.5%
Test accuracy: 90.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

train_dataset_sm = train_dataset[0:300,:]
train_labels_sm = train_labels[0:300,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_sm.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_sm[offset:(offset + batch_size), :]
    batch_labels = train_labels_sm[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3485.348145
Minibatch accuracy: 8.6%
Validation accuracy: 41.4%
Minibatch loss at step 500: 20.951054
Minibatch accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 0.382695
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 1500: 0.220552
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 2000: 0.206564
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.202562
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 3000: 0.201302
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Test accuracy: 84.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
def calculate_logits_with_dropout(tf_train_dataset, weights0, weights1, biases0, biases1):
    hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights0) + biases0),0.5)
    output_layer = tf.matmul(hidden_layer, weights1) + biases1
    return output_layer
    
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights0 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases0 = tf.Variable(tf.zeros([1024]))
  weights1 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  beta = tf.constant(0.01)
  
  # Training computation.
  logits = calculate_logits_with_dropout(tf_train_dataset, weights0, weights1, biases0, biases1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
       + beta*tf.nn.l2_loss(weights0)+ beta*tf.nn.l2_loss(weights1)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(calculate_logits(tf_valid_dataset, weights0, weights1, biases0, biases1))
  test_prediction = tf.nn.softmax(calculate_logits(tf_test_dataset, weights0, weights1, biases0, biases1)) 
    

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3688.353027
Minibatch accuracy: 9.4%
Validation accuracy: 32.6%
Minibatch loss at step 500: 21.410248
Minibatch accuracy: 84.4%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.978898
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.810252
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 0.836503
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 0.892515
Minibatch accuracy: 78.9%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 0.926770
Minibatch accuracy: 77.3%
Validation accuracy: 83.2%
Test accuracy: 90.3%


---

The extreme overfit case

---

In [12]:
num_steps = 3001


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_sm.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_sm[offset:(offset + batch_size), :]
    batch_labels = train_labels_sm[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3643.762207
Minibatch accuracy: 10.9%
Validation accuracy: 23.7%
Minibatch loss at step 500: 21.057367
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 0.412197
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.246956
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.230596
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.215013
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.219238
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Test accuracy: 84.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
import math as math

def calculate_logits_w_dropout(tf_train_dataset, weights, biases):
    next_layer = tf_train_dataset
    for i in range(len(weights)):        
        next_layer = tf.nn.relu(tf.matmul(next_layer, weights[i]) + biases[i])
    return next_layer

def calculate_logits(tf_train_dataset, weights, biases):
    next_layer = tf_train_dataset
    for i in range(len(weights)):        
        next_layer = tf.nn.relu(tf.matmul(next_layer, weights[i]) + biases[i])
    return next_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_nodes = [image_size*image_size, 2048, 512, num_labels]
  weights = []
  biases = []
  for i in range(3):
      weights.append(tf.Variable(tf.truncated_normal([num_nodes[i], num_nodes[i+1]], stddev=0.5*math.sqrt((1/num_nodes[i+1])))))
      biases.append(tf.Variable(tf.zeros(num_nodes[i+1])))

  # Training computation.
  logits = calculate_logits_w_dropout(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # Decaying learning rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  start_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(calculate_logits(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(calculate_logits(tf_test_dataset, weights, biases))

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.311966
Minibatch accuracy: 10.2%
Validation accuracy: 27.5%
Minibatch loss at step 500: 0.410233
Minibatch accuracy: 86.7%
Validation accuracy: 85.3%
Minibatch loss at step 1000: 0.302545
Minibatch accuracy: 88.3%
Validation accuracy: 87.6%
Minibatch loss at step 1500: 0.348209
Minibatch accuracy: 89.8%
Validation accuracy: 88.0%
Minibatch loss at step 2000: 0.340073
Minibatch accuracy: 91.4%
Validation accuracy: 89.0%
Minibatch loss at step 2500: 0.479061
Minibatch accuracy: 84.4%
Validation accuracy: 89.4%
Minibatch loss at step 3000: 0.413051
Minibatch accuracy: 83.6%
Validation accuracy: 88.8%
Test accuracy: 94.3%
